In [1]:
# import the libraries needed
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Preparing the train and test dataset
---
- We have obtained the dataset from https://www.openpowerlifting.org/
- From there, we randomised the data rows in the original CSV file and took out a total of 3,000 data points to be used for our train and test dataset

In [2]:
train_test_combined = pd.read_csv('../datasets/train_test_combined.csv')
train_test_combined.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alana Carrasco,F,SBD,Raw,29.5,24-34,24-39,FR-O,68.60,72,...,Yes,USA,AZ,USAPL,IPF,27/8/2017,USA,AZ,NaN,Sun Devil Classic
1,Even Dysjaland,M,SBD,Raw,19.5,20-23,19-23,Juniors 19-23,94.65,105,...,Yes,NaN,NaN,NSF,IPF,13/6/2015,Norway,NaN,Ganddal,Klubbstevne
2,Alessandro Favorito,M,SBD,Single-ply,39.0,35-39,24-39,Open,89.40,90,...,Yes,Italy,NaN,FIPL,IPF,6/11/2004,Italy,NaN,Marina Di Carrara,Coppa Italia A Squadre
3,Sydney Martinez,F,SBD,Single-ply,NaN,NaN,NaN,Girls,55.16,56,...,Yes,USA,NaN,THSWPA,NaN,25/1/2014,USA,TX,JUNIOR HIGH,CARRIZO SPRINGS INVITATIONAL
4,Navy Villar,F,SBD,Single-ply,NaN,NaN,NaN,Girls,55.61,56,...,Yes,NaN,NaN,THSWPA,NaN,18/1/2018,USA,TX,Sadler,S&S Lady Rams Meet


In [8]:
# Split combined dataset into train and test
from sklearn.model_selection import train_test_split

train,test = train_test_split(train_test_combined, test_size = 0.25, random_state = 0)
train.to_csv('../datasets/train.csv', index = False)
test.to_csv('../datasets/test.csv', index = False)

print(train.shape)
print(test.shape)

(3000, 41)
(1000, 41)


In [9]:
# Import train dataset into a dataframe
train_df = pd.read_csv('../datasets/train.csv')
train_df.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Hanne Bingle,F,SBD,Wraps,61.0,60-64,60-69,M5,73.70,75,...,NaN,UK,NaN,GPC,GPC,1/9/2020,Slovakia,NaN,Trnava,European Championships
1,Mohamed Reda,M,SBD,Raw,NaN,NaN,NaN,Open,99.58,105,...,Yes,Egypt,NaN,AfricanPF,IPF,20/10/2018,Morocco,NaN,Meknes,African & Arab Powerlifting Championships
2,Pauline Reeves,F,SBD,Single-ply,NaN,45-49,NaN,Open,129.30,90+,...,Yes,England,NaN,BAWLA,IPF,26/4/2003,UK,NaN,Livingstone,British Masters' Championships
3,Edie Montalvo,F,SBD,Single-ply,NaN,NaN,NaN,Girls,71.12,75,...,Yes,NaN,NaN,THSWPA,NaN,10/2/2018,USA,TX,Weslaco,Weslaco Girls Invitational Powerlifting Meet
4,Faith Bailey,F,SBD,Single-ply,NaN,NaN,NaN,Girls,78.83,82.3,...,Yes,NaN,NaN,THSWPA,NaN,7/3/2020,USA,TX,Bay City,Region 4 Division 3 Meet


## Data Preprocessing
---

### Removing disqualified lifters

First, find out if any lifters were disqualified (DQ) or did not turn up for the competition (NS). Proceed to remove them from our dataset.

In [13]:
train_df['Place'].unique()

array(['1', '3', 'DQ', '7', '9', '2', '4', '6', 'NS', '5', '14', '11',
       '10', '8', '13', '12', '18', '15', '22', '24', '32', '26', '17',
       '19', '16', 'G', '28', 'DD', '23', '51', '21', '84', '20', '29'],
      dtype=object)

In [14]:
df = train_df.drop(train_df[
    (train_df.Place == 'DQ') |
    (train_df.Place == 'NS') |
    (train_df.Place == 'DD')
].index)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2814 non-null   object 
 1   Sex               2814 non-null   object 
 2   Event             2814 non-null   object 
 3   Equipment         2814 non-null   object 
 4   Age               1613 non-null   float64
 5   AgeClass          1884 non-null   object 
 6   BirthYearClass    1721 non-null   object 
 7   Division          2812 non-null   object 
 8   BodyweightKg      2775 non-null   float64
 9   WeightClassKg     2788 non-null   object 
 10  Squat1Kg          1284 non-null   float64
 11  Squat2Kg          1270 non-null   float64
 12  Squat3Kg          1230 non-null   float64
 13  Squat4Kg          11 non-null     float64
 14  Best3SquatKg      2787 non-null   float64
 15  Bench1Kg          1285 non-null   float64
 16  Bench2Kg          1277 non-null   float64


After dropping disqualified and no-show lifters from the dataset, we are left with only 2814 values from the original 3000 values. 

## Filling Missing Values (Age)
First, use the AgeClass column to fill in missing values for Age column. We will assume that the lifter's age is the maximum age within the AgeClass.

In [7]:
df[df['Age'].isna() & df['AgeClass'].notna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
21,Melissa Salinas,F,SBD,Single-ply,NaN,16-17,14-18,Girls,73.39,75,...,Yes,NaN,NaN,THSWPA,NaN,17/1/2015,USA,TX,Skidmore Texas,2nd Annual Skidmore Invitational
26,Amy Hoffman,F,SBD,Single-ply,NaN,45-49,40-49,Masters 45-49,75.00,75,...,NaN,NaN,NaN,USPF,NaN,5/12/2015,USA,WV,Parkersburg,Region 2 Championships
33,Nick Cooper,M,SBD,Single-ply,NaN,13-15,14-18,M-T1,75.00,75,...,Yes,USA,NaN,USAPL,IPF,23/3/2002,USA,FL,Lakeland,Frank Kostyo Memorial
40,Atle Andersen,M,SBD,Single-ply,NaN,24-34,24-39,Open,74.60,75,...,Yes,NaN,NaN,NSF,IPF,24/2/1991,Norway,NaN,Lykkeberghallen,Seriestevne
56,Diane Farrar,F,SBD,Single-ply,NaN,45-49,40-49,F-M1b,60.00,60,...,Yes,NaN,NaN,USAPL,IPF,15/5/2004,USA,RI,Warwick,New England States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,J. Shelton,M,SBD,Single-ply,NaN,16-17,14-18,Teen 14-16,67.50,67.5,...,Yes,Canada,NaN,ADFPA,NaN,23/3/1990,USA,MS,Jackson,Mississippi Teenage
3943,Sanjuana Delgado,F,SBD,Single-ply,NaN,18-19,NaN,Girls,55.97,56,...,Yes,NaN,NaN,THSWPA,NaN,5/3/2016,USA,TX,Academy,Region 2 Division 3
3945,Ramirez Jr,M,SBD,Single-ply,NaN,40-44,40-49,Masters 40-44,90.00,90,...,NaN,NaN,NaN,USPF,IPF,14/5/1993,USA,TX,Irving,Masters Nationals
3965,Kayla Anderson #3,F,SBD,Single-ply,NaN,18-19,NaN,Girls,93.71,100,...,Yes,NaN,NaN,THSWPA,NaN,24/1/2019,USA,TX,Henderson,Lion Invitational


In [179]:
def newString(g):
    ## Function that returns the last two characters of a str as an int. 
    if type(g) == str:
        return int(g[-2:])
    return g

In [180]:
## First, fill the values using the AgeClass Strings
df['Age'].fillna(df.AgeClass, inplace=True)

In [181]:
## Use the newString function to replace the Strings with the maximum age 
df['Age'] = df['Age'].apply(newString)

In [183]:
#compare indexes against the DataFrame printed earlier to ensure the replacement was done correctly
df.loc[29]

Name                       Ashley Gilbert
Sex                                     F
Event                                 SBD
Equipment                      Single-ply
Age                                  19.0
AgeClass                            18-19
BirthYearClass                        NaN
Division                            Girls
BodyweightKg                        79.56
WeightClassKg                        82.3
Squat1Kg                              NaN
Squat2Kg                              NaN
Squat3Kg                              NaN
Squat4Kg                              NaN
Best3SquatKg                       102.06
Bench1Kg                              NaN
Bench2Kg                              NaN
Bench3Kg                              NaN
Bench4Kg                              NaN
Best3BenchKg                        72.57
Deadlift1Kg                           NaN
Deadlift2Kg                           NaN
Deadlift3Kg                           NaN
Deadlift4Kg                       

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2828 non-null   object 
 1   Sex               2828 non-null   object 
 2   Event             2828 non-null   object 
 3   Equipment         2828 non-null   object 
 4   Age               2228 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2118 non-null   object 
 7   Division          2828 non-null   object 
 8   BodyweightKg      2822 non-null   float64
 9   WeightClassKg     2813 non-null   object 
 10  Squat1Kg          1085 non-null   float64
 11  Squat2Kg          1077 non-null   float64
 12  Squat3Kg          1054 non-null   float64
 13  Squat4Kg          5 non-null      float64
 14  Best3SquatKg      1871 non-null   float64
 15  Bench1Kg          1712 non-null   float64
 16  Bench2Kg          1703 non-null   float64


Now the age column only contains numeric float64 datatypes, which shows that we have successfully replaced all the strings after the fillna() operation.

However, there are still some missing values, where both Age and AgeClass data was missing.

We can fill the remaining missing values for Age with the median age of their respective divisions.

In [184]:
df[df['Age'].isna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
2,Paige Shuemake,F,SBD,Single-ply,NaN,NaN,NaN,Girls,54.25,56,...,Yes,NaN,NaN,THSWPA,NaN,31/1/2015,USA,TX,Malakoff,Malakoff
3,Pål Nilsen,M,SBD,Single-ply,NaN,NaN,NaN,Open,89.20,90,...,Yes,Norway,NaN,NSF,IPF,6/6/1998,Norway,NaN,Oslo,KM Oslo-Akershus
8,Tori Rash,F,SBD,Single-ply,NaN,NaN,NaN,Girls,59.96,60.1,...,Yes,NaN,NaN,THSWPA,NaN,24/1/2019,USA,TX,Needville,Needville 1-24-19
9,Stina Christiane Valmestad,F,B,Raw,NaN,NaN,NaN,Open,73.23,84,...,Yes,NaN,NaN,NSF,IPF,15/12/2018,Norway,NaN,Sande,Klubbstevne
10,Kyle Monhollen,M,SBD,Raw,NaN,NaN,NaN,MR-O,104.20,105,...,Yes,NaN,NaN,USAPL,IPF,22/8/2015,USA,OH,NaN,Future Summer Shakedown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,Alejandra Fierro,F,SBD,Single-ply,NaN,NaN,NaN,Girls,51.94,51.9,...,Yes,NaN,NaN,THSWPA,NaN,7/2/2019,USA,TX,Plainview,Plainview Triangular
2986,Tor Erik Rødsdalen,M,SBD,Single-ply,NaN,NaN,NaN,Open,89.40,90,...,Yes,NaN,NaN,NSF,IPF,16/2/2002,Norway,NaN,Brumunddal,B-og Veteran NM
2995,Roberto Bettati,M,SBD,Single-ply,NaN,NaN,40-49,Masters 1,88.40,90,...,Yes,Italy,NaN,FIPL,IPF,29/4/2006,Italy,NaN,Nettuno,Campionato Italiano Assoluto
2996,Lachlan Hodgetts,M,B,Raw,NaN,NaN,NaN,Open,72.70,74,...,Yes,Canada,BC,CPU,IPF,23/6/2018,Canada,BC,Surrey,BCPA Provincial Championship


In [217]:
t = df.groupby('Division')['Age'].median()
t ## Series showing median age of each division

Division
13-14               13.5
17-19               18.0
40-49               45.0
45-49               48.0
55-59               57.0
                    ... 
Youth 10-15         13.5
Youth 11 & Under     6.0
Youth 11-12         11.0
sen                  NaN
Юниорки              NaN
Name: Age, Length: 369, dtype: float64

In [216]:
df['Age'] = df['Age'].fillna(df.groupby('Division')['Age'].transform('median'))

##stackoverflow magic https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group


In [221]:
## cell for checking
print(df.loc[8]['Age']) ## value in the Age column after the fillna()
print(t[df.loc[8]['Division']]) ## median value of the division

19.0
19.0


In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2828 non-null   object 
 1   Sex               2828 non-null   object 
 2   Event             2828 non-null   object 
 3   Equipment         2828 non-null   object 
 4   Age               2793 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2118 non-null   object 
 7   Division          2828 non-null   object 
 8   BodyweightKg      2822 non-null   float64
 9   WeightClassKg     2813 non-null   object 
 10  Squat1Kg          1085 non-null   float64
 11  Squat2Kg          1077 non-null   float64
 12  Squat3Kg          1054 non-null   float64
 13  Squat4Kg          5 non-null      float64
 14  Best3SquatKg      1871 non-null   float64
 15  Bench1Kg          1712 non-null   float64
 16  Bench2Kg          1703 non-null   float64


Drop the remaining values NA values for age

In [227]:
df.dropna(subset=['Age'], inplace=True)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2793 non-null   object 
 1   Sex               2793 non-null   object 
 2   Event             2793 non-null   object 
 3   Equipment         2793 non-null   object 
 4   Age               2793 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2114 non-null   object 
 7   Division          2793 non-null   object 
 8   BodyweightKg      2790 non-null   float64
 9   WeightClassKg     2780 non-null   object 
 10  Squat1Kg          1074 non-null   float64
 11  Squat2Kg          1066 non-null   float64
 12  Squat3Kg          1044 non-null   float64
 13  Squat4Kg          4 non-null      float64
 14  Best3SquatKg      1850 non-null   float64
 15  Bench1Kg          1696 non-null   float64
 16  Bench2Kg          1687 non-null   float64


# Filling Missing Values (Bodyweight)

In [231]:
df[df.BodyweightKg.isna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
1086,Richard Nowazek,M,SBD,Single-ply,51.5,50-54,50-59,Open,NaN,140+,...,Yes,Canada,BC,CPU,IPF,6/4/2002,Canada,BC,Vancouver,BC Provincial Championships
1597,Madison Neesmith,F,B,Raw,20.0,20-23,19-23,JR,NaN,82.5+,...,Yes,USA,NC,WNPF,NaN,6/10/2018,USA,NaN,NaN,World Tournament Of Championships
1605,Jay Bakke,M,SBD,Multi-ply,53.0,50-54,50-59,M_MEM_3_AAPF,NaN,NaN,...,Yes,USA,NaN,APF,WPC,17/7/2010,USA,MT,NaN,Big Sky State Games


We can use the weightclasses to fill in the first 2 missing bodyweight values. We assume they hold the minimum weight in the particular weightclass

Weight classes can be specified as a maximum or as a minimum. Maximums are specified by just the number, for example `90` means "up to (and including) 90kg." minimums are specified by a `+` to the right of the number, for example `90+` means "above (and excluding) 90kg.

In [238]:
df['BodyweightKg'].fillna(df.WeightClassKg, inplace=True)

In [241]:
df['BodyweightKg'] = df['BodyweightKg'].apply(lambda f: float(f[:-1]) if type(f) == str else f)

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2793 non-null   object 
 1   Sex               2793 non-null   object 
 2   Event             2793 non-null   object 
 3   Equipment         2793 non-null   object 
 4   Age               2793 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2114 non-null   object 
 7   Division          2793 non-null   object 
 8   BodyweightKg      2792 non-null   float64
 9   WeightClassKg     2780 non-null   object 
 10  Squat1Kg          1074 non-null   float64
 11  Squat2Kg          1066 non-null   float64
 12  Squat3Kg          1044 non-null   float64
 13  Squat4Kg          4 non-null      float64
 14  Best3SquatKg      1850 non-null   float64
 15  Bench1Kg          1696 non-null   float64
 16  Bench2Kg          1687 non-null   float64


The final missing bodyweight value belongs to a Male in the 50-54 AgeClass. We can use the median value of that AgeClass to fill the remaining missing value.

In [246]:
df.groupby(['AgeClass']).median()

,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
AgeClass,,,,,,,,,,,,,,,,,,,,,
13-15,14.50,58.700,98.75,96.50,83.75,65.0,104.330,52.500,57.50,37.500,...,108.750,118.75,101.05,-105.00,115.00,220.00,211.270,213.220,208.210,47.030
16-17,16.50,70.220,115.00,122.50,96.50,NaN,126.005,70.000,62.50,-35.875,...,128.750,138.75,130.00,-226.25,137.50,300.00,274.870,273.400,256.490,53.130
18-19,18.50,73.900,137.50,147.50,112.50,NaN,142.500,87.500,90.00,-60.000,...,170.000,173.75,125.00,-125.00,154.22,331.12,306.720,302.640,280.990,59.310
20-23,21.50,81.200,170.00,170.00,118.75,165.0,186.250,105.000,107.50,56.250,...,185.000,190.00,130.00,327.50,212.50,422.50,337.770,334.795,315.670,71.550
24-34,28.00,83.000,147.50,157.50,127.50,241.0,182.500,120.000,115.00,-55.000,...,190.000,190.00,126.25,125.00,210.00,320.00,299.400,298.430,270.620,73.910
35-39,37.00,86.910,140.00,117.50,117.50,NaN,193.750,119.000,90.00,-52.500,...,177.500,156.25,142.50,137.50,210.00,267.50,181.200,181.590,173.640,71.165
40-44,42.00,91.900,165.00,158.76,117.50,NaN,190.000,132.500,117.50,-67.500,...,200.000,203.75,120.00,NaN,218.75,240.00,151.900,154.030,146.390,70.120
45-49,47.00,89.050,123.75,151.25,132.50,NaN,182.500,130.000,105.00,-50.000,...,150.000,160.00,157.50,188.00,192.64,225.00,160.365,158.555,142.755,67.210
5-Dec,11.00,39.900,30.00,45.00,-38.75,NaN,25.000,23.750,20.00,-22.500,...,42.215,63.50,-40.00,-86.18,35.00,35.00,44.490,46.740,46.350,21.615


In [247]:
df.BodyweightKg.fillna(82.45, inplace=True)